# Feature engineering

In [1]:
# Traitement de données
import pandas as pd
import numpy as np

In [2]:
X = pd.read_csv("X_train_NHkHMNU.csv")
y = pd.read_csv("y_train_ZAN5mwg.csv")

# Merge both datasets
df = pd.concat([X, y], axis=1)

# Drop the second-to-last column if needed
df = df.drop(df.columns[-2], axis=1)
df = df.sort_values("DAY_ID")

Change Nan Values

Transformation "ReLU"

In [ ]:
from typing import List

def AddSeuilColumn(df: pd.DataFrame, column_name: str, seuil: float, way: str):
    message = column_name + "_Threshold_" + str(seuil)
    if way == "sup":
        df[message] = df[column_name].where(df[column_name] >= seuil, 0)
    else:
        df[message] = df[column_name].where(df[column_name] <= seuil, 0)